In [18]:
## merge bhc and discharge instruction results

import pandas as pd

# results_bhc_fullpath = r"C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\CODE\discharge_me\data\scoring\submission_01\submission.csv.csv"
# results_discharge_instructions_fullpath = r"C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\CODE\discharge_me\data\scoring\submission_01\submission.csv.csv"
# targets_fullpath = r"C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\CODE\discharge_me\data\scoring\submission_01\submission.csv.csv"

results_bhc_fullpath = r"results_bhc.csv"
results_discharge_instructions_fullpath = r"results_discharge_instructions.csv"
targets_fullpath = r"discharge_target.csv"


# Load the data
results_bhc_df = pd.read_csv(results_bhc_fullpath)
results_discharge_instructions_df = pd.read_csv(results_discharge_instructions_fullpath).drop_duplicates(subset='hadm_id', keep='first')
targets_df = pd.read_csv(targets_fullpath)

print(results_bhc_df.head())
print(results_bhc_df.shape)
print(results_bhc_df.hadm_id.dtypes)
print(results_discharge_instructions_df.head())
print(results_discharge_instructions_df.shape)
print(results_discharge_instructions_df.hadm_id.dtypes)
print(targets_df.head())
print(targets_df.shape)
print(targets_df.hadm_id.dtypes)


# Identify missing 'hadm_ids' in both dataframes
missing_in_discharge = set(targets_df['hadm_id']) - set(results_discharge_instructions_df['hadm_id'])
missing_in_bhc = set(targets_df['hadm_id']) - set(results_bhc_df['hadm_id'])

# # Print missing 'hadm_ids' with corresponding row numbers
if missing_in_discharge:
    print("Missing 'hadm_ids' in results_discharge_instructions:")
    for id in missing_in_discharge:
        row_numbers = results_bhc_df.index[results_bhc_df['hadm_ids'] == id].tolist()
        print(f"hadm_id {id} missing at row numbers in results_bhc: {row_numbers}")

if missing_in_bhc:
    print("Missing 'hadm_ids' in results_bhc:")
    for id in missing_in_bhc:
        row_numbers = results_discharge_instructions_df.index[results_discharge_instructions_df['hadm_ids'] == id].tolist()
        print(f"hadm_id {id} missing at row numbers in results_discharge_instructions: {row_numbers}")

# check if duplicates 'hadm_id'"Final_training_BHC (1).ipynb" in both dataframes
duplicates_in_discharge = results_discharge_instructions_df[results_discharge_instructions_df.duplicated(subset=['hadm_id'])]

# Merging DataFrames
merged_df = pd.merge(results_bhc_df, results_discharge_instructions_df, on='hadm_id', how='inner')

# Display the merged DataFrame
print("Merged DataFrame:")
print(merged_df.head())
print(merged_df.shape)


# Writing the merged DataFrame to a CSV file
file_path = 'merged_results.csv'  # Specify your file path and name here
merged_df.to_csv(file_path, index=False)  # index=False means do not write row numbers
print(f"Data has been written to {file_path}")

    hadm_id                              brief_hospital_course
0  24962904  Brief Hospital Course\nThe patient with a hist...
1  22774359  Brief Hospital Course\nThe patient with a hist...
2  29323205  Brief Hospital Course\nThe patient with a hist...
3  20459702  Brief Hospital Course\nThe patient with a hist...
4  28755331  Brief Hospital Course\nThe patient with a hist...
(10962, 2)
int64
    hadm_id                             discharge_instructions
0  24962904  Discharge Instructions ---\nDear Ms. ___,\n\nI...
1  22774359  Discharge Instructions ---\nDear you,\n\nIt wa...
2  29323205  Discharge Instructions ---\nDear you,\n\nIt wa...
3  20459702  Discharge Instructions ---\nDear Ms. ___,\n\nI...
4  28755331  Discharge Instructions ---\nDear Mr. ___,\n\nI...
(10962, 2)
int64
          note_id   hadm_id  \
0  10001884-DS-35  24962904   
1  10003019-DS-22  22774359   
2   10003299-DS-7  29323205   
3   10003502-DS-7  20459702   
4  10004322-DS-21  28755331   

                       

In [22]:
import pandas as pd
import re # Regular expressions

# results_fullpath = r"C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\CODE\discharge_me\data\scoring\submission_01\submission.csv.csv"
results_fullpath = r"merged_results.csv"

# Load the data
results_df = pd.read_csv(results_fullpath)

# Display the first few rows

print(results_df.head())
print(results_df.shape)
print(results_df.columns)


processed_results_df = pd.DataFrame(results_df)
# clean up discharge_instructions
# Replace "Dear you," with "Dear ___,"
processed_results_df['discharge_instructions'] = processed_results_df['discharge_instructions'].str.replace('Dear you,', 'Dear ___,',  flags=re.IGNORECASE, regex=False)
processed_results_df['discharge_instructions'] = processed_results_df['discharge_instructions'].str.replace(r'\[Briefly [^\]]*\]', '___',  flags=re.IGNORECASE,regex=True)
processed_results_df['discharge_instructions'] = processed_results_df['discharge_instructions'].str.replace(r'\[Summarize  [^\]]*\]', '___',  flags=re.IGNORECASE,regex=True)
processed_results_df['discharge_instructions'] = processed_results_df['discharge_instructions'].str.replace(r'I am sorry, but as an AI language model *', '___',  flags=re.IGNORECASE,regex=True)
processed_results_df['discharge_instructions'] = processed_results_df['discharge_instructions'].apply(
        lambda x: '___error___' if x.startswith("I am sorry, but as an AI language model,") else x
    )

# Replace strings matching "[Briefly ?]" with "___"
processed_results_df['brief_hospital_course'] = processed_results_df['brief_hospital_course'].str.replace(r'\[Briefly [^\]]*\]', '___',  flags=re.IGNORECASE,regex=True)
# Replace strings matching "[summarize ?]" with "___"
processed_results_df['brief_hospital_course'] = processed_results_df['brief_hospital_course'].str.replace(r'\[Summarize [^\]]*\]', '___',  flags=re.IGNORECASE,regex=True)

print(processed_results_df.head())
print(processed_results_df.shape)   
print(processed_results_df.columns)


## Writing the processed DataFrame to a CSV file
file_path = 'post_processed_results.csv'  # Specify your file path and name here
processed_results_df.to_csv(file_path, index=False)  # index=False means do not write row numbers
print(f"Data has been written to {file_path}")
print(processed_results_df.head())
print(processed_results_df.shape)   


    hadm_id                              brief_hospital_course  \
0  24962904  Brief Hospital Course\nThe patient with a hist...   
1  22774359  Brief Hospital Course\nThe patient with a hist...   
2  29323205  Brief Hospital Course\nThe patient with a hist...   
3  20459702  Brief Hospital Course\nThe patient with a hist...   
4  28755331  Brief Hospital Course\nThe patient with a hist...   

                              discharge_instructions  
0  Discharge Instructions ---\nDear Ms. ___,\n\nI...  
1  Discharge Instructions ---\nDear you,\n\nIt wa...  
2  Discharge Instructions ---\nDear you,\n\nIt wa...  
3  Discharge Instructions ---\nDear Ms. ___,\n\nI...  
4  Discharge Instructions ---\nDear Mr. ___,\n\nI...  
(10962, 3)
Index(['hadm_id', 'brief_hospital_course', 'discharge_instructions'], dtype='object')
    hadm_id                              brief_hospital_course  \
0  24962904  Brief Hospital Course\nThe patient with a hist...   
1  22774359  Brief Hospital Course\nThe pati

In [23]:
print(processed_results_df.head())


    hadm_id                              brief_hospital_course  \
0  24962904  Brief Hospital Course\nThe patient with a hist...   
1  22774359  Brief Hospital Course\nThe patient with a hist...   
2  29323205  Brief Hospital Course\nThe patient with a hist...   
3  20459702  Brief Hospital Course\nThe patient with a hist...   
4  28755331  Brief Hospital Course\nThe patient with a hist...   

                              discharge_instructions  
0  Discharge Instructions ---\nDear Ms. ___,\n\nI...  
1  Discharge Instructions ---\nDear ___,\n\nIt wa...  
2  Discharge Instructions ---\nDear ___,\n\nIt wa...  
3  Discharge Instructions ---\nDear Ms. ___,\n\nI...  
4  Discharge Instructions ---\nDear Mr. ___,\n\nI...  
